In [1]:
import pickle
import glob
from collections import Counter, defaultdict
import json
import newsplease
import minerva
import fastText as ft
import pandas as pd

In [2]:
m = minerva.Minerva()

In [14]:
pd.set_option('display.max_colwidth', 100)
m._Minerva__risk_data

,id,short_id,long_label,label,type,keywords,description,category,vector
0,ASSET,ASSET,Asset bubbles in a major economy,Asset bubbles in a major economy,Risk,"financial crises, banking crises, banking panics, recession, economic bubble, asset bubble, spec...","Unsustainably overpriced assets such as commodities, housing, shares, etc. in a major economy or...",Economic,"[-0.08505873, 0.019556038, 0.0442262, 0.014739149, 0.0007510881, -0.06862396, -0.03375716, 0.032..."
1,DEFLATION,DEFLATION,Deflation in a major economy,Deflation,Risk,"deflation, growth deflation, hoarding deflation, credit deflation, debt deflation, macroeconomic...",Prolonged near-zero inflation or deflation in a major economy or region,Economic,"[-0.10740217, 0.068878375, 0.016524509, 0.0138784, -0.02097979, -0.09326975, 0.011826792, 0.0045..."
2,FINANCIALMECH,FINANCIALMECH,Failure of a major financial mechanism or institution,Failure of financial mechanism or institution,Risk,"bank failure, illiquidity, illiquid, insolvent, global failure, financial crises, savings lost, ...",Collapse of a financial institution and/or malfunctioning of a financial system that impacts the...,Economic,"[-0.072594784, 0.072460495, 0.03638007, 0.02123094, -0.05338822, -0.07405142, -0.028528351, 0.01..."
3,CRITICALINFRA,CRITICALINFRA,Failure/shortfall of critical infrastructure,Failure of critical infrastructure,Risk,"failure of critical infrastructure, energy crises, cascading effect, transportation crises, comm...","Failure to adequately invest in, upgrade and/or secure infrastructure networks (e.g. energy, tra...",Economic,"[-0.062981285, 0.052816898, 0.086468466, 1.9597355e-05, -0.09430506, -0.0052090567, -0.037004247..."
4,FISCALCRISES,FISCALCRISES,Fiscal crises in key economies,Fiscal crises,Risk,"debt crises, sovereign debt, liquidity crises, macroeconomic vulnerability, macroeconomic debt",Excessive debt burdens that generate sovereign debt crises and/or liquidity crises,Economic,"[-0.11107647, 0.0429964, 0.02748474, -0.0013690824, -0.04949342, -0.06971179, -0.018248117, 0.05..."
5,UNEMPLOYMENT,UNEMPLOYMENT,High structural unemployment or underemployment,Unemployment or underemployment,Risk,"unemployment, job, jobs, employment crises, layoff, job crises, mass layoff, employment reductio...",A sustained high level of unemployment or underutilization of the productive capacity of the emp...,Economic,"[-0.08285182, 0.07296477, 0.023937372, -0.024935925, 0.007642543, -0.031445444, 0.033579215, 0.0..."
6,ILLICITTRADE,ILLICITTRADE,"Illicit trade (e.g. illicit financial flows, tax evasion, human trafficking, organized crime, etc.)",Illicit trade,Risk,"illicit trade, illicit financial flows, tax evasion, human trafficking, organized crime, counter...","Large-scale activities outside the legal framework such as illicit financial flows, tax evasion,...",Economic,"[-0.08996466, 0.040528156, 0.0019822915, -0.0074719884, -0.0030418602, -0.010007762, 0.005345972..."
7,ENERGY,ENERGY,Severe energy price shock (increase or decrease),Energy price shock,Risk,"energy price, energy crises, energy finance, energy access, efficient energy supply, electrical ...",Significant energy price increases or decreases that place further economic pressures on highly ...,Economic,"[-0.1019072, 0.10087774, 0.072774485, 0.013391656, -0.032783166, 0.0071929083, -0.026238142, 0.0..."
8,INFLATION,INFLATION,Unmanageable inflation,Unmanageable inflation,Risk,"inflation, high inflation, price increase, price levels, economy price, housing prices, oil pric...",Unmanageable increases in the general price levels of goods and services in key economies,Economic,"[-0.14934477, 0.07180878, 0.07808182, -0.029089276, -0.0045678145, -0.04971775, -0.004117151, -1..."
9,WEATHEREVENTS,WEATHEREVENTS,"Extreme weather events (e.g. floods, storms, etc.)",Extreme weather events,Risk,"extreme weather, flood, storm, natural dissaster, tropical storm, cyclone, drought, heat weave, ...","Major property, infrastructure and/or environmen

In [36]:
pd.set_option('display.max_colwidth', -1)
m._Minerva__risk_data[m._Minerva__risk_data['id'] == 'MANMADECATA']['keywords']

13    oil spill, radioactive contamination, man-made damage, man-made disaster, enviromental crime, salinization, garbage, chemical plant explosion, chemical contamination, radiological contamination, nuclear explosion, oil fire, methane gas release, mud flow, asbestos contamination, toxic waste, mercury poisoning, e-waste, electronic waste, cyanide contamination, shrinking sea, industrial disaster, toxic chemical, smog, ariborne pollutants, manmade disaster, human overpopulation, biocapacity, carrying capacity, exploitation of natural resources, industrialization, land degradation, land reclamation, optimum population, overshoot population, population density, population dynamics, population growth, projections of population growth, total fertility rate, urbanization, waste, water conflict, water scarcity, overdrafting, hydrology, environmental impacts of reservoirs, tile drainage, hydrology, flooding, landslide, intensive farming, agricultural subsidy, environmental effects of meat p

---

In [4]:
infos_dir = "risk_info_extracted/"

In [7]:
info_files = sorted(glob.glob(infos_dir + "/*.pkl"))

In [9]:
infos = []
for f in info_files:
    with open(f, "rb") as fin:
        infos.append(pickle.load(fin))

In [10]:
len(infos)

28

In [12]:
len(infos[0])

7842

In [15]:
json_infos = []
for json_info in infos:
    json_infos.extend(json_info)

In [24]:
len(json_infos)

251933

In [25]:
pickle.dump(json_infos, open("json_infos.pkl", "wb"))

In [26]:
json_infos[0]

{'category': 'Societal',
 'category_prob': 0.26844912265080634,
 'date_publish': datetime.datetime(2018, 8, 5, 14, 3, 51),
 'entities': {'Asaduzzaman Khan',
  'Bangladesh',
  'Bangladesh Nationalist Party',
  'Bangladeshi',
  'Chowdhury',
  'DHAKA',
  'Dhaka',
  'Dhaka Bangladesh',
  'Dhaka University',
  'Hasina',
  'Marcia Bernicat',
  'Mohammad Ponir Hossain',
  'REUTERS',
  'Serajul Islam Chowdhury',
  'The U S',
  'U S'},
 'image_url': 'https://s4.reutersmedia.net/resources/r/?m=02&d=20180805&t=2&i=1290449244&w=1200&r=LYNXMPEE7408V',
 'risk': 'WATER',
 'risk_sim': 0.6379263913153423,
 'source_domain': 'www.reuters.com',
 'title': 'Bangladesh police fire tear gas to clear protesters blocking traffic',
 'url': 'https://www.reuters.com/article/us-bangladesh-protests/bangladesh-police-fire-tear-gas-to-clear-protesters-blocking-traffic-idUSKBN1KQ098',
 'week': '31',
 'year': '2018'}

In [59]:
time2risk = dict()
for info in json_infos:
    year_week = (int(info['year']), int(info['week']))
    
    if year_week not in time2risk:
        time2risk[year_week] = dict()
        
    if info['risk'] not in time2risk[year_week]:
        time2risk[year_week][info['risk']] = Counter()
    
    time2risk[year_week][info['risk']].update(info['entities'])

In [60]:
time2risk_items = list(time2risk.items())

In [61]:
time2risk_items[0]

((2011, 32),
 {'FOOD': Counter({'Africa': 1,
           'Braintree': 1,
           'Brooks Newmark': 1,
           'Central Suffolk': 1,
           'Dan Poulter': 1,
           'Dr Poulter': 1,
           'Education': 1,
           'Elizabeth': 1,
           'Girubuntu School': 1,
           'Gorleston': 1,
           'House': 1,
           'Kigali': 1,
           'Michael Ellis': 1,
           'Morrisons': 1,
           'NHS': 1,
           'Netherlands': 1,
           'Norman Lamb': 1,
           'Norman Lamb MP': 1,
           'North Ipswich': 1,
           'North Norfolk': 1,
           'North West Norfolk': 1,
           'Northampton': 1,
           'Richard Bacon': 1,
           'Rwanda': 1,
           'South Norfolk': 1,
           'VSO': 1,
           'Westminster': 1,
           'the James Paget Hospital': 1}),
  'SOCIALINSTABILITY': Counter({'Daily Express': 1,
           'Guardian': 1,
           'Hackney': 1,
           'Ken Livingstone': 1,
           'London': 1,
        

In [66]:
time2risk_2018 = {k:v for k,v in time2risk.items() if 2018 == k[0] and k[1] <= 31}

In [67]:
print(len(time2risk))
print(len(time2risk_2018))

409
31


In [68]:
for k in [k for k in time2risk.keys() if 2018 != k[0]][0:5]:
    print(k)

(2011, 32)
(2015, 29)
(2014, 47)
(2010, 49)
(2012, 43)


In [69]:
for k in sorted(time2risk_2018.keys()):
    print(k, len(time2risk_2018[k]))

(2018, 1) 19
(2018, 2) 19
(2018, 3) 20
(2018, 4) 19
(2018, 5) 20
(2018, 6) 21
(2018, 7) 20
(2018, 8) 20
(2018, 9) 21
(2018, 10) 21
(2018, 11) 20
(2018, 12) 19
(2018, 13) 21
(2018, 14) 20
(2018, 15) 20
(2018, 16) 20
(2018, 17) 20
(2018, 18) 20
(2018, 19) 20
(2018, 20) 20
(2018, 21) 20
(2018, 22) 20
(2018, 23) 20
(2018, 24) 19
(2018, 25) 20
(2018, 26) 21
(2018, 27) 19
(2018, 28) 20
(2018, 29) 21
(2018, 30) 19
(2018, 31) 21


In [71]:
pickle.dump(time2risk_2018, open("time2risk_2018.pkl", "wb"))

In [138]:
def jaccard_sim(a, b):
    return len(a.intersection(b)) / len(a.union(b))

# def compute_link_weights(risks, topn=10):
#     risk_keys = sorted(list(risks.keys()))
#     links = []
#     for i in range(len(risk_keys) - 1):
#         ri = risk_keys[i]
#         rj = risk_keys[i+1]
#         wij = jaccard_sim(set(risks[ri].keys()), set(risks[rj].keys()))
        
#         entities_in_common = risks[ri] & risks[rj]
#         entities_topn = entities_in_common.most_common(topn)
        
#         links.append((ri, rj, wij, entities_topn))
#     return links
        
def compute_link_weights(risks):
    risk_keys = sorted(list(risks.keys()))
    links = []
    for i in range(0, len(risk_keys)):
        for j in range(i+1, len(risk_keys)):
            ri = risk_keys[i]
            rj = risk_keys[j]
            wij = jaccard_sim(set(risks[ri].keys()), set(risks[rj].keys()))
            links.append((ri, rj, wij))
    return links
    

In [139]:
time2risk_links = dict()
for t in time2risk_2018.keys():
    time2risk_links[t] = compute_link_weights(time2risk_2018[t])
    

In [140]:
time2risk_links_items = list(time2risk_links.items())

In [141]:
sorted(time2risk_links_items, key=lambda x: x[0][1])[0:2]

[((2018, 1),
  [('ASSET', 'CRITICALINFRA', 0.033268101761252444),
   ('ASSET', 'DEFLATION', 0.02781740370898716),
   ('ASSET', 'ENERGY', 0.11505143475906876),
   ('ASSET', 'FAILUREURBAN', 0.037028969723371816),
   ('ASSET', 'FINANCIALMECH', 0.09610291335603481),
   ('ASSET', 'FOOD', 0.028470840348997398),
   ('ASSET', 'ILLICITTRADE', 0.08231545406266595),
   ('ASSET', 'INFECTIOUSDISEASES', 0.012265978050355068),
   ('ASSET', 'INFLATION', 0.1569713758079409),
   ('ASSET', 'INTERSTATE', 0.036577400391900716),
   ('ASSET', 'MIGRATION', 0.051668726823238566),
   ('ASSET', 'NATIONALGOV', 0.054589567327133035),
   ('ASSET', 'SOCIALINSTABILITY', 0.028729752770673488),
   ('ASSET', 'STATECOLLAPSE', 0.05398457583547558),
   ('ASSET', 'TERRORIST', 0.03898405197873597),
   ('ASSET', 'UNEMPLOYMENT', 0.11066559743384122),
   ('ASSET', 'WATER', 0.04482705134170932),
   ('ASSET', 'WMD', 0.031207598371777476),
   ('CRITICALINFRA', 'DEFLATION', 0.028070175438596492),
   ('CRITICALINFRA', 'ENERGY', 0.02

In [123]:
#pickle.dump(time2risk_links_items, open("time2risk_links_items.pkl", "wb"))

In [142]:
link_items = sorted(time2risk_links_items, key=lambda x: x[0][1])

In [127]:
minerva_entities = json.load(open("entities_minerva.json", "r"))

In [153]:
risk2info = {e['id']: e for e in minerva_entities}

In [158]:
risk2info

{'ASSET': {'category': 'Economic',
  'category_idx': 0,
  'description': 'Unsustainably overpriced assets such as commodities, housing, shares, etc. in a major economy or region',
  'id': 'ASSET',
  'keywords': 'financial crises, banking crises, banking panics, recession, economic bubble, asset bubble, speculative bubble, market bubble, price bubble, financial bubble, speculative mania, currency crises, sovereign debt, currency crises, financial sector instability, macroeconomic vulnerability, macroeconomic debt',
  'label': 'Asset bubbles in a major economy',
  'long_label': 'Asset bubbles in a major economy',
  'risk_idx': 0,
  'type': 'Risk'},
 'BIODIVERSITY': {'category': 'Environmental',
  'category_idx': 1,
  'description': 'Irreversible consequences for the environment, resulting in severely depleted resources for humankind as well as industries',
  'id': 'BIODIVERSITY',
  'keywords': 'biodiversity loss, loss of biodiversity, extinction, extinction of species, ecological impact,

In [149]:
def compute_topn_links(risk_links, topn=5):
    risk2links = dict()
    for ri, rj, wij in risk_links:
        if ri not in risk2links:
            risk2links[ri] = Counter()

        if rj not in risk2links:
            risk2links[rj] = Counter()

        risk2links[ri][rj] += wij
        risk2links[rj][ri] += wij
        
    topn_links = dict()
    for risk in risk2links.keys():
        topn_links[risk] = risk2links[risk].most_common(topn)
    return topn_links
        
            

In [208]:
topn_links = compute_topn_links(time2risk_links_items[4][1], topn=7)

In [209]:
topn_links

{'ASSET': [('ILLICITTRADE', 0.14090343970161626),
  ('INFLATION', 0.13178294573643412),
  ('ENERGY', 0.11318482841661649),
  ('FINANCIALMECH', 0.0942501481920569),
  ('UNEMPLOYMENT', 0.08822003113648158),
  ('STATECOLLAPSE', 0.056481979558902634),
  ('NATIONALGOV', 0.05355612682090831)],
 'CRITICALINFRA': [('WMD', 0.04132231404958678),
  ('INTERSTATE', 0.03436426116838488),
  ('STATECOLLAPSE', 0.03306878306878307),
  ('UNEMPLOYMENT', 0.03160270880361174),
  ('ILLICITTRADE', 0.0274869109947644),
  ('NATIONALGOV', 0.027375201288244767),
  ('INFECTIOUSDISEASES', 0.023762376237623763)],
 'DEFLATION': [('WMD', 0.018315018315018316),
  ('UNEMPLOYMENT', 0.0175),
  ('TERRORIST', 0.017142857142857144),
  ('INTERSTATE', 0.015151515151515152),
  ('INFECTIOUSDISEASES', 0.014598540145985401),
  ('INFLATION', 0.014285714285714285),
  ('STATECOLLAPSE', 0.01188707280832095)],
 'ENERGY': [('INFLATION', 0.13430180847909873),
  ('ASSET', 0.11318482841661649),
  ('ILLICITTRADE', 0.10868298368298368),
  ('

In [210]:
def generate_data_graph(topn_links, risk2info):
    json_data = list()
    for risk_i in topn_links.keys():
        risk_record = dict()
        risk_info = risk2info[risk_i]
        category = risk_info['category']
        category_idx = risk_info['category_idx']
        risk_idx = risk_info['risk_idx']
        name = risk_info['label']
        info = risk_info
        links = []
        for risk_j, wij in topn_links[risk_i]:
            links.append(("{}.{}".format(risk2info[risk_j]['category'], risk2info[risk_j]['label']), wij))
        
        risk_record = {
            'group': category_idx,
            'id': risk_idx,
            'links': links,
            'name': "{}.{}".format(category, name),
            'info': info
        }
        
        json_data.append(risk_record)
        
    return json_data

In [211]:
json_str = json.dumps(generate_data_graph(topn_links, risk2info), indent=4)

In [212]:
print(json_str)

[
    {
        "info": {
            "description": "Large-scale activities outside the legal framework such as illicit financial flows, tax evasion, human trafficking, counterfeiting and/or organized crime that undermine social interactions, regional or international collaboration, and global growth",
            "keywords": "illicit trade, illicit financial flows, tax evasion, human trafficking, organized crime, counterfitting, crime illegal drugs, crime caterls, crime, crime networks, drug trade, trade policy and integration, trade linkages spillovers and connectivity, economic trade dispute, trade balance, imact of trade, natural resources trafficking, illicit financial flows",
            "category": "Economic",
            "category_idx": 0,
            "label": "Illicit trade",
            "type": "Risk",
            "id": "ILLICITTRADE",
            "long_label": "Illicit trade (e.g. illicit financial flows, tax evasion, human trafficking, organized crime, etc.)",
            

In [221]:
# fetch risk categories:
cats = Counter()
for _, links in time2risk_links_items:
    for a, b, _ in links:
        cats[risk2info[a]['category']] += 1
        cats[risk2info[b]['category']] += 1

In [219]:
# [{u'id': u'0', u'name': u'Economic Risks'},
# {u'id': u'1', u'name': u'Environmental Risks'},
# {u'id': u'2', u'name': u'Geopolitical Risks'},
# {u'id': u'3', u'name': u'Societal Risks'},
# {u'id': u'4', u'name': u'Technological Risks'}]

In [222]:
cats

Counter({'Economic': 5136,
         'Geopolitical': 3024,
         'Societal': 3534,
         'Technological': 100})

---

In [5]:
articles = None
with open("201808_articles/articles_with_text.pkl", "rb") as fin:
    articles = pickle.load(fin)

In [6]:
len(articles)

25865

In [12]:
articles[0].get_dict()

{'authors': [],
 'date_download': datetime.datetime(2018, 8, 3, 2, 44, 53, tzinfo=tzutc()),
 'date_modify': None,
 'date_publish': datetime.datetime(2018, 8, 3, 2, 39, 58),
 'description': 'Rais wa Zimbabwe Emmerson Mnangagwa ameshinda uchaguzi wa urais nchini Zimbabwe kwa mujibu wa tume ya uchaguzi ya nchi hiyo.',
 'filename': 'https%3A%2F%2Fwww.bbc.com%2Fswahili%2Fhabari-45054310.json',
 'image_url': 'https://ichef.bbci.co.uk/news/1024/branded_swahili/68B1/production/_102810862__102809937_048385698-1.jpg',
 'language': 'sw',
 'localpath': None,
 'source_domain': 'www.bbc.com',
 'text': 'Rais wa Zimbabwe Emmerson Mnangagwa ameshinda uchaguzi wa urais nchini Zimbabwe kwa mujibu wa tume ya uchaguzi ya nchi hiyo.\nKatika majimbo 10 yote yalioyotangazwa, Mnangagwa ameshinda kwa asilimia 50.8 ya kura kwa asilimia 44.3 alizopata mgombea wa chama cha upinzani Nelson Chamise.\nHaki miliki ya picha Getty Images Image caption Zimbabwe Emmerson Mnangagwa\nPolisi waliwaondoa upinzani wakati wa ku

In [20]:
articles_en = [a for a in [x for x in articles if x.get_dict()['title'] is not None] if m.detect_language(a.get_dict()['title'].replace("\n", " "))[0] == 'en']

In [21]:
len(articles), len(articles_en)

(25865, 21060)

In [3]:
#pickle.dump(articles_en, open("201808_articles/articles_with_text_en.pkl", "wb"))
articles_en = pickle.load(open("201808_articles/articles_with_text_en.pkl", "rb"))

In [4]:
len(articles_en)

21060

In [17]:
articles_en[0].get_dict()

{'authors': ['Harry Sumnall',
  'Prof In Substance Use',
  'John Moores University'],
 'date_download': datetime.datetime(2018, 8, 3, 2, 45, 43, tzinfo=tzutc()),
 'date_modify': None,
 'date_publish': datetime.datetime(2018, 8, 3, 2, 44, 1),
 'description': 'Who actually uses a drug blamed for violence, environmental and social problems?',
 'filename': 'https%3A%2F%2Fwww.bbc.com%2Fnews%2Fuk-45036469.json',
 'image_url': 'https://ichef.bbci.co.uk/news/1024/branded_news/11FBC/production/_102806637_dance.jpg',
 'language': 'en',
 'localpath': None,
 'source_domain': 'www.bbc.com',
 'text': 'Image copyright Getty Images\n"Middle-class" cocaine users have been accused by police and politicians of fuelling crime. But who actually uses the drug?\nThe problem, says the UK\'s most senior police officer, is a group of middle-class people who worry about "global warming and fair trade... but think there is no harm in taking a bit of cocaine".\nThe comments, from Metropolitan Police Commissioner C

In [5]:
article = articles_en[0]
doc_vec = m.get_vector(article.get_dict()['text'])
lang, proba = m.detect_language(article.get_dict()['title'])
print(lang)
is_risk = m.is_risk(doc_vec)
is_risk

en


True

In [6]:
print(m.transform(article))

{'category': 'Economic', 'entities': {('Metropolitan Police', 'ORG'), ('BBC', 'ORG'), ('Harry Sumnall', 'PERSON'), ("the Home Office's Crime Survey for England", 'ORG'), ('UK', 'GPE'), ('London', 'GPE'), ('England', 'GPE'), ('Sadiq Khan', 'PERSON'), ('the Misuse of Drugs', 'ORG'), ('PA Image', 'ORG'), ('Cressida Dick', 'PERSON'), ('the Public Health Institute Liverpool John Moores University', 'ORG'), ('Wales', 'GPE'), ('the Office for National Statistics ONS', 'ORG'), ('Getty Images Middle', 'PERSON'), ("Cosmopolitan'", 'ORG'), ('the UK Advisory Council', 'ORG'), ('Londoners', 'NORP')}, 'date_publish': datetime.datetime(2018, 8, 3, 2, 44, 1), 'title': 'Cocaine The drug that is more than a middle class problem', 'image_url': 'https://ichef.bbci.co.uk/news/1024/branded_news/11FBC/production/_102806637_dance.jpg', 'source_domain': 'www.bbc.com', 'year': '2018', 'risk_sim': 0.740582524892557, 'risk': 'ILLICITTRADE', 'url': 'https://www.bbc.com/news/uk-45036469', 'week': '31'}


In [25]:
ft_lang_model = ft.load_model("lid.176.ftz")

In [26]:
ft_lang_model.predict(article.get_dict()['title'])

(('__label__en',), array([0.96333271]))

In [5]:
%%time
risk_infos = []
count = 0
n_articles = len(articles_en)
for article in articles_en:
    risk_info = m.transform(article)
    if risk_info:
        risk_infos.append((article.get_dict(), risk_info))
    count += 1
    if count % 1000 == 0:
        print("processing {}/{} articles. Successfully extracted: {}".format(count, n_articles, len(risk_infos)))
pickle.dump(risk_infos, open('20180816_risk_infos.pkl', 'wb'))
print("done.")

ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
/datax1/minerva/CC-NEWS/minerva/toolbox.py:13: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' 

processing 1000/21060 articles. Successfully extracted: 670


ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'


processing 2000/21060 articles. Successfully extracted: 1465


ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/m

...

processing 19000/21060 articles. Successfully extracted: 13466


ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/m

processing 20000/21060 articles. Successfully extracted: 14105


ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/minerva.py", line 244, in transform
    year = article_info['date_publish'].strftime("%Y")
AttributeError: 'NoneType' object has no attribute 'strftime'
ERROR:minerva:'NoneType' object has no attribute 'strftime'
Traceback (most recent call last):
  File "/datax1/minerva/CC-NEWS/minerva/m

processing 21000/21060 articles. Successfully extracted: 14832
done.
CPU times: user 8h 45min 13s, sys: 19h 30min 16s, total: 1d 4h 15min 30s
Wall time: 59min 28s


---

In [12]:
%%time
articles_with_is_risk_flag = []
count = 0
n_articles = len(articles_en)
for article in [a.get_dict() for a in articles_en]:
    is_risk_from_title, _ = m.is_risk(m.get_vector(article['title']))
    is_risk_from_text, _ = m.is_risk(m.get_vector(article['text']))
    article['is_risk_from_title'] = is_risk_from_title
    article['is_risk_from_text'] = is_risk_from_text
    articles_with_is_risk_flag.append(article)
    count += 1
    if count % 1000 == 0:
        print("processing {}/{} articles. Passed: {}".format(count, n_articles, len(articles_with_is_risk_flag)))
print("done.")

processing 1000/21060 articles. Passed: 1000
processing 2000/21060 articles. Passed: 2000
processing 3000/21060 articles. Passed: 3000
processing 4000/21060 articles. Passed: 4000
processing 5000/21060 articles. Passed: 5000
processing 6000/21060 articles. Passed: 6000
processing 7000/21060 articles. Passed: 7000
processing 8000/21060 articles. Passed: 8000
processing 9000/21060 articles. Passed: 9000
processing 10000/21060 articles. Passed: 10000
processing 11000/21060 articles. Passed: 11000
processing 12000/21060 articles. Passed: 12000
processing 13000/21060 articles. Passed: 13000
processing 14000/21060 articles. Passed: 14000
processing 15000/21060 articles. Passed: 15000
processing 16000/21060 articles. Passed: 16000
processing 17000/21060 articles. Passed: 17000
processing 18000/21060 articles. Passed: 18000
processing 19000/21060 articles. Passed: 19000
processing 20000/21060 articles. Passed: 20000
processing 21000/21060 articles. Passed: 21000
done.
CPU times: user 9h 34min 

In [15]:
pickle.dump(articles_with_is_risk_flag, open('20180816_articles_with_is_risk_flag.pkl', 'wb'))

In [16]:
no_risk = [a for a in articles_with_is_risk_flag if a['is_risk_from_title'] == False]
yes_risk = [a for a in articles_with_is_risk_flag if a['is_risk_from_title'] == True]

In [17]:
len(yes_risk), len(no_risk)

(15024, 6036)

In [18]:
list(enumerate([(a['title'], a['is_risk_from_title'], a['is_risk_from_text']) for a in no_risk[0:25]]))

[(0, ("'A gimmicky tournament': AFLW draw under fire", False, True)),
 (1,
  ('Baltimore Ravens LB Kamalei Correa sacks Chicago Bears QB Chase Daniel',
   False,
   True)),
 (2,
  ('Watch Ravens LB Kamalei Correa intercept Bears QB Chase Daniel',
   False,
   True)),
 (3, ('Watch Ravens QB Lamar Jackson throw his 1st NFL TD', False, True)),
 (4,
  ('Ravens Robert Griffin III hits Maxx Williams for TD in HOF Game',
   False,
   True)),
 (5,
  ('Ravens QB Lamar Jackson throws 1st career NFL interception in HOF game',
   False,
   True)),
 (6,
  ('Josh Woodrum, not Lamar Jackson plays after RG3 for Ravens vs. Bears',
   False,
   True)),
 (7, ('Watch Chuck Clark get an INT off a tipped pass', False, True)),
 (8,
  ("Ray Lewis does famous 'Squirrel Dance' to enter Hall of Fame Game",
   False,
   True)),
 (9,
  ('Kerri Walsh Jennings plans to chase gold in Tokyo, retire', False, True)),
 (10,
  ('Scherzer fans 10, Nationals cruise to 10-4 win over Reds', False, True)),
 (11, ('Franco’s 3-r

In [19]:
no_risk[19]

{'authors': ['Gavino Borquez'],
 'date_download': datetime.datetime(2018, 8, 3, 3, 16, 51, tzinfo=tzutc()),
 'date_modify': None,
 'date_publish': datetime.datetime(2018, 8, 3, 1, 32, 4),
 'description': 'Los Angeles Chargers WR Dylan Cantrell is the real deal.',
 'filename': 'https%3A%2F%2Fchargerswire.usatoday.com%2F2018%2F08%2F02%2Fchargers-wr-dylan-cantrell-is-the-real-deal%2F.json',
 'image_url': 'https://usatchargerswire.files.wordpress.com/2018/08/usatsi_11009320.jpg?w=1024&h=576&crop=1',
 'is_risk_from_text': True,
 'is_risk_from_title': False,
 'language': 'en',
 'localpath': None,
 'source_domain': 'chargerswire.usatoday.com',
 'text': "Despite not being a significant need for the Chargers, selecting a wide receiver was something the team needed to address with the rocky 2017 season from WR Mike Williams. That is why the selection of former Texas Tech WR Dylan Cantrell made complete sense, and he has been off to a great start in training camp.\nGeno Smith dropping a dime to D

In [49]:
categories_info = m._Minerva__categories_info

In [50]:
categories = m._Minerva__categories

In [51]:
doc_vec = m.get_vector(no_risk[19]['title'])

In [59]:
is_risk, category_risk_probas = m.is_risk(doc_vec)

In [60]:
is_risk, category_risk_probas

(True,
 {'Economic': 0.17263663926568157,
  'Environmental': 1.9716101205773607e-06,
  'Geopolitical': 0.1930683446079942,
  'Societal': 0.3443055917691066,
  'Technological': 0.03554732421128643})

In [69]:
def is_risk(m, doc_vec, threshold=0.5, max_deviations=3):
    """check if the text is considered a risk in at least one category, if so, we process the article"""
    is_risk = False
    category_risk_probas = dict()
    for category in m._Minerva__categories:
        categories_info = m._Minerva__categories_info[category]
        risk_proba = m.risk_proba(doc_vec,
                                     categories_info['centroid'],
                                     categories_info['median'],
                                     categories_info['mad'],
                                     max_deviation=max_deviations)
        category_risk_probas[category] = risk_proba
        is_risk = is_risk or (risk_proba > threshold)
    return is_risk, category_risk_probas

In [95]:
print(no_risk[19]['title'])
doc_vec = m.get_vector(no_risk[19]['title'])
is_risk(m, doc_vec, threshold=0.5, max_deviations=5)

Macron and May to discuss Brexit at president’s summer residence


(True,
 {'Economic': 0.6065767027951383,
  'Environmental': 1.4568154275093546e-05,
  'Geopolitical': 0.6387182406346673,
  'Societal': 0.795081905017748,
  'Technological': 0.21404792479022594})

In [96]:
print(no_risk[20]['title'])
doc_vec = m.get_vector(no_risk[20]['title'])
doc_vec = m.get_vector(no_risk[20]['title'])

Kerri Walsh Jennings plans to chase gold in Tokyo, retire


In [97]:
is_risk(m, doc_vec, threshold=0.5, max_deviations=5)

(False,
 {'Economic': 0.29794394367347166,
  'Environmental': 4.062034756343838e-07,
  'Geopolitical': 0.22867371417245203,
  'Societal': 0.2803551564084976,
  'Technological': 0.07263414165810911})

---

In [28]:
#risk_infos = pickle.load(open('20180816_risk_infos.pkl', 'rb'))

In [9]:
len(risk_infos)

14872

In [8]:
def count_articles_per_category(risk_infos):
    counts = Counter()
    for _,r in risk_infos:
        if r['category'] is None:
            print(r)
        counts[r['category']] += 1
    return counts

In [9]:
def count_articles_per_risk(risk_infos):
    counts = Counter()
    for _,r in risk_infos:
        if r['risk'] is None:
            print(r)
        counts[r['risk']] += 1
    return counts

In [10]:
counts = count_articles_per_category(risk_infos)

In [13]:
len(counts), counts

(5,
 Counter({'Economic': 5618,
          'Environmental': 867,
          'Geopolitical': 5245,
          'Societal': 1693,
          'Technological': 1447}))

In [15]:
risk_counts = count_articles_per_risk(risk_infos)

In [16]:
len(risk_counts), risk_counts

(30,
 Counter({'ASSET': 261,
          'BIODIVERSITY': 18,
          'CRITICALINFO': 674,
          'CRITICALINFRA': 492,
          'CYBERATTACKS': 16,
          'DATAFRAUD': 723,
          'DEFLATION': 4,
          'ENERGY': 1552,
          'FAILURECLIMATE': 242,
          'FAILUREURBAN': 141,
          'FINANCIALMECH': 324,
          'FISCALCRISES': 1,
          'FOOD': 964,
          'ILLICITTRADE': 1234,
          'INFECTIOUSDISEASES': 47,
          'INFLATION': 595,
          'INTERSTATE': 127,
          'MANMADECATA': 115,
          'MIGRATION': 114,
          'NATIONALGOV': 1735,
          'NATURALCATA': 22,
          'REGIONALGOV': 6,
          'SOCIALINSTABILITY': 182,
          'STATECOLLAPSE': 2630,
          'TECHADV': 34,
          'TERRORIST': 418,
          'UNEMPLOYMENT': 1155,
          'WATER': 245,
          'WEATHEREVENTS': 470,
          'WMD': 329}))

In [18]:
len(risk_infos[0])

2

In [19]:
risk_infos[0][1]

{'category': 'Economic',
 'date_publish': datetime.datetime(2018, 8, 3, 2, 44, 1),
 'entities': {('BBC', 'ORG'),
  ("Cosmopolitan'", 'ORG'),
  ('Cressida Dick', 'PERSON'),
  ('England', 'GPE'),
  ('Getty Images Middle', 'PERSON'),
  ('Harry Sumnall', 'PERSON'),
  ('London', 'GPE'),
  ('Londoners', 'NORP'),
  ('Metropolitan Police', 'ORG'),
  ('PA Image', 'ORG'),
  ('Sadiq Khan', 'PERSON'),
  ('UK', 'GPE'),
  ('Wales', 'GPE'),
  ("the Home Office's Crime Survey for England", 'ORG'),
  ('the Misuse of Drugs', 'ORG'),
  ('the Office for National Statistics ONS', 'ORG'),
  ('the Public Health Institute Liverpool John Moores University', 'ORG'),
  ('the UK Advisory Council', 'ORG')},
 'image_url': 'https://ichef.bbci.co.uk/news/1024/branded_news/11FBC/production/_102806637_dance.jpg',
 'risk': 'ILLICITTRADE',
 'risk_sim': 0.740582524892557,
 'source_domain': 'www.bbc.com',
 'title': 'Cocaine The drug that is more than a middle class problem',
 'url': 'https://www.bbc.com/news/uk-45036469',
